In [ ]:
import pickle
with open('/workspace/persistent/text_representation_kim.pkl', 'rb') as file:
    # Load the contents of the PKL file
    tk = pickle.load(file)

In [ ]:
import pickle
with open('/workspace/persistent/graph_representation_kim.pkl', 'rb') as file:
    # Load the contents of the PKL file
    gk = pickle.load(file)

In [ ]:
import torch
post_representation=[]
for i in range(len(gk)):
    concatenated_tensor = torch.cat([tk[i][0][0], gk[i][0][0]], dim=0)
    post_representation.append(concatenated_tensor)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(770, 32)
        self.fc2 = nn.Linear(32, 64)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc3 = nn.Linear(64, 128)
        self.fc4 = nn.Linear(128, 2)

    def forward(self, x):
        x = x.unsqueeze(0)  # Add a batch dimension
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.dropout1(x)
        x = self.fc3(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc4(x)
        output = F.log_softmax(x, dim=1)
        return output

post_prediction=[]
my_nn = Net()
# Forward pass to get the output
for i in range(len(post_representation)):
    output = my_nn(post_representation[i]).argmax()
    post_prediction.append(output)

In [ ]:
len(post_representation)

In [ ]:
import pickle
with open('/workspace/persistent/post_prediction_kim.pkl', 'wb') as file:
    pickle.dump(post_prediction, file)

In [ ]:
with open('/workspace/persistent/test_split_kim.pkl', 'rb') as file:
    # Load the contents of the PKL file
    test_split = pickle.load(file)
test_split['id']=0
test_split['id']=range(len(test_split))
test_split = test_split.set_index(test_split['id'])

In [ ]:
true=[]
pred=[]
for i in range(len(test_split)):
    true.append(float(test_split['sponsored'][i]))
for i in range(len(test_split)):
    pred.append(float(post_prediction[i]))


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

 # Predicted values

accuracy = accuracy_score(true, pred)
precision = precision_score(true, pred)
recall = recall_score(true, pred)
f1 = f1_score(true, pred)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 score:", f1)

In [ ]:
import numpy as np
from scipy.special import softmax

def listMLE_loss(pred_scores, true_scores):
    pred_probs = softmax(pred_scores)
    sorted_probs = np.sort(pred_probs)[::-1]
    cum_probs = np.cumsum(sorted_probs)
    loss = np.sum(np.log(1 + cum_probs)) - np.sum(np.log(1 + sorted_probs))
    return loss

def gradient_listMLE_loss(pred_scores, true_scores):
    pred_probs = softmax(pred_scores)
    sorted_probs = np.sort(pred_probs)[::-1]
    cum_probs = np.cumsum(sorted_probs)
    gradient = pred_probs - cum_probs / (1 + cum_probs)
    return gradient

def optimize_listMLE_loss(pred_scores, true_scores, learning_rate=0.1, num_iterations=100):
    pred_scores = np.array(pred_scores)
    true_scores = np.array(true_scores)

    for _ in range(num_iterations):
        gradient = gradient_listMLE_loss(pred_scores, true_scores)
        pred_scores -= learning_rate * gradient

    return pred_scores

# Example usage
pred_scores = pred
true_scores = true

optimized_scores = optimize_listMLE_loss(pred_scores, true_scores)
loss = listMLE_loss(optimized_scores, true_scores)

print("Optimized Scores:", optimized_scores)
print("ListMLE Loss:", loss)


In [ ]:
def calculate_metrics(scores, truth, threshold):
    rounded_scores = [0 if score <= threshold else 1 for score in scores]

    tp = sum(1 for rounded_score, true_value in zip(rounded_scores, truth) if rounded_score == 1 and true_value == 1)
    fp = sum(1 for rounded_score, true_value in zip(rounded_scores, truth) if rounded_score == 1 and true_value == 0)
    fn = sum(1 for rounded_score, true_value in zip(rounded_scores, truth) if rounded_score == 0 and true_value == 1)

    accuracy = sum(1 for rounded_score, true_value in zip(rounded_scores, truth) if rounded_score == true_value) / len(truth)
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    return accuracy, precision, recall, f1

accuracy, precision, recall, f1 = calculate_metrics(optimized_scores, true, 0.5)
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 score:", f1)